In [1]:
import pandas as pd

In [2]:
requests_df = pd.read_csv("../data/requests.csv")
vlans_df = pd.read_csv("../data/vlans.csv")
output = []
file_name = "../data/output.csv"

sorted_vlans_df = vlans_df.sort_values(by=['vlan_id', 'device_id', 'primary_port'])
redundant_vlans = sorted_vlans_df.groupby(['device_id','vlan_id'])['primary_port'].filter(lambda g: g.nunique() > 1).reset_index()
redundant_vlan_indexes = redundant_vlans['index'].tolist()



def trim_sorted_vlans_df():
    useless_rows = sorted_vlans_df[(sorted_vlans_df.primary_port == 0) & (~sorted_vlans_df.index.isin(redundant_vlan_indexes))].index.tolist()
    sorted_vlans_df.drop(useless_rows, inplace=True)



In [3]:
def drop_unavailable_redundant_vlans(rows_to_drop):
    sorted_vlans_df.drop(rows_to_drop, inplace=True)
    del redundant_vlan_indexes[:2]
    return True
    
def drop_unavailable_non_redundant_vlans(rows_to_drop):
    crossing_redundant_vlans_index = [i for i,x in enumerate(redundant_vlan_indexes) if x == rows_to_drop[-1]]
    if(not crossing_redundant_vlans_index):
        sorted_vlans_df.drop(rows_to_drop, inplace=True)
        return True
    if(crossing_redundant_vlans_index % 2 == 0):
        rows_to_drop.append(redundant_vlan_indexes[crossing_redundant_vlans_index+1])
        sorted_vlans_df.drop(rows_to_drop, inplace=True)
        del redundant_vlan_indexes[crossing_redundant_vlans_index:crossing_redundant_vlans_index+2]
    else:
        rows_to_drop.append(redundant_vlan_indexes[crossing_redundant_vlans_index-1])
        sorted_vlans_df.drop(rows_to_drop, inplace=True)
        del redundant_vlan_indexes[crossing_redundant_vlans_index-1:crossing_redundant_vlans_index+1]
    return True
    
def populate_output(lowest_vlans_indexes, request_id):
    for index in lowest_vlans_indexes:
        device_id = sorted_vlans_df.loc[index]['device_id']
        primary_port = sorted_vlans_df.loc[index]['primary_port']
        vlan_id = sorted_vlans_df.loc[index]['vlan_id']
        output.append((request_id, device_id, primary_port, vlan_id))
    
def get_lowest_available_vlan_with_redundancy(request_id):
    lowest_vlans_indexes = redundant_vlan_indexes[:2]
    populate_output(lowest_vlans_indexes, request_id)
    drop_unavailable_redundant_vlans(lowest_vlans_indexes)

def get_lowest_available_vlan_no_redundancy(request_id):
    lowest_vlans_indexes = sorted_vlans_df[(sorted_vlans_df.primary_port == 1)].head(1).index.tolist()
    populate_output(lowest_vlans_indexes, request_id)
    drop_unavailable_non_redundant_vlans(lowest_vlans_indexes)

trim_sorted_vlans_df()

for request_id, redundant in requests_df.itertuples(index=False):
    if redundant:
        get_lowest_available_vlan_with_redundancy(request_id)
    else:
        get_lowest_available_vlan_no_redundancy(request_id)

output_df = pd.DataFrame(output, columns=('request_id', 'device_id', 'primary_port', 'vlan_id'))
output_df

output_df.to_csv(file_name, encoding='utf-8', index=False)
